In [4]:
import pandas as pd
from surprise import Dataset, Reader, SVD, accuracy, NMF, KNNWithMeans
from surprise.model_selection import train_test_split, GridSearchCV, cross_validate

In [2]:
train = pd.read_csv('/Users/Stefano_1/Documents/Data/ml2017-fall-hw5/train.csv')
test = pd.read_csv('/Users/Stefano_1/Documents/Data/ml2017-fall-hw5/test.csv')
#movies = pd.read_csv('/Users/Stefano_1/Documents/Data/ml2017-fall-hw5/movies.csv', delimiter= '::', encoding= 'latin1')
#users = pd.read_csv('/Users/Stefano_1/Documents/Data/ml2017-fall-hw5/users.csv', delimiter='::')

In [3]:
reader = Reader(rating_scale= (1,5))
data = Dataset.load_from_df(train[['UserID', 'MovieID', 'Rating']], reader)

In [5]:
algs = {'svd': SVD(),
        'NMF': NMF(),
        'KNNWithMeans': KNNWithMeans()}

In [6]:
results = []

for name, model in algs.items():
    
    res = cross_validate(model, data, measures=['rmse'], cv = 5, verbose= True)

    res = res['test_rmse'].mean()
    results.append(res)

results


Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8767  0.8796  0.8819  0.8826  0.8807  0.8803  0.0021  
Fit time          5.76    5.85    5.86    5.86    5.85    5.84    0.04    
Test time         1.05    0.95    0.94    0.94    1.06    0.99    0.06    
Evaluating RMSE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9186  0.9185  0.9182  0.9194  0.9186  0.9187  0.0004  
Fit time          7.21    9.65    7.26    9.49    7.40    8.20    1.12    
Test time         0.96    1.12    0.81    0.98    1.10    0.99    0.11    
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing 

[0.8803122149373538, 0.918663586647367, 0.9312480598413766]

In [7]:
trainset, testset = train_test_split(data, test_size=0.3)

In [8]:
mod = KNNWithMeans()

mod.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [10]:
test_pred = mod.test(testset)
accuracy.rmse(test_pred)

RMSE: 0.9350


0.9349917533618609

In [15]:
test.dtypes

TestDataID    int64
UserID        int64
MovieID       int64
dtype: object

In [17]:
predictions = []

for _,row in test.iterrows():

    user = row['UserID']
    movie = row['MovieID']
    #id = row['TestDataID']

    y_pred = mod.predict(user, movie)
    predictions.append((row['TestDataID'], y_pred.est))

submission = pd.DataFrame(predictions, columns=['TestDataID', 'Rating'])

submission['Rating'] = submission.Rating.round()


In [18]:
submission.to_csv('/Users/Stefano_1/Documents/Data/Kaggle Movie Rec/knnwm1.csv', index_label= 'id', index = False)